In [1]:
import numpy as np
from math import sqrt
from sklearn import preprocessing

In [2]:
data = np.loadtxt("data/spambase.data", dtype='f', delimiter=',')

print (data)
print (data.dtype)        

[[0.000e+00 6.400e-01 6.400e-01 ... 6.100e+01 2.780e+02 1.000e+00]
 [2.100e-01 2.800e-01 5.000e-01 ... 1.010e+02 1.028e+03 1.000e+00]
 [6.000e-02 0.000e+00 7.100e-01 ... 4.850e+02 2.259e+03 1.000e+00]
 ...
 [3.000e-01 0.000e+00 3.000e-01 ... 6.000e+00 1.180e+02 0.000e+00]
 [9.600e-01 0.000e+00 0.000e+00 ... 5.000e+00 7.800e+01 0.000e+00]
 [0.000e+00 0.000e+00 6.500e-01 ... 5.000e+00 4.000e+01 0.000e+00]]
float32


Isso mostra que teremos que tratar os dados antes de rodar __Online SVM via Online Gradient Descent__

Temos ao menos duas etapas:
1 - Transformar os valores de classe ( variável target ) de 0, 1 para -1 ,1
2 - Temos que fazer uma normalização dos parâmetros. Pois alguns dados são percentuais e estão no range [0, 100] e outros podem atingir valores maiores como por exemplo 15841. Não queremos que um atributo tenha uma importância maior que outro por ser representado em uma escala diferente. Vamos fazer uma normalização __hard__ que é mapear o maior valor de uma determinada feature para 1 e dividir os outros valores dessa feature específica pelo maior valor encontrado.

Se a normalização hard se mostrar ruim na prática, podemos pensar em outras normalizações. ( Soft usando mediana e desvio-padrão )..



In [3]:
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(data)

for entry in x_scaled:
    if entry[57] < 1:
        entry[57] = -1.0

In [4]:
x_scaled

array([[ 0.00000000e+00,  4.48179282e-02,  1.25490189e-01, ...,
         6.00720895e-03,  1.74873751e-02,  1.00000000e+00],
       [ 4.62555066e-02,  1.96078438e-02,  9.80392173e-02, ...,
         1.00120148e-02,  6.48358613e-02,  1.00000000e+00],
       [ 1.32158585e-02,  0.00000000e+00,  1.39215678e-01, ...,
         4.84581478e-02,  1.42550513e-01,  1.00000000e+00],
       ...,
       [ 6.60792962e-02,  0.00000000e+00,  5.88235334e-02, ...,
         5.00600727e-04,  7.38636404e-03, -1.00000000e+00],
       [ 2.11453736e-01,  0.00000000e+00,  0.00000000e+00, ...,
         4.00480581e-04,  4.86111175e-03, -1.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.27450973e-01, ...,
         4.00480581e-04,  2.46212143e-03, -1.00000000e+00]], dtype=float32)

Breve descrição de Online SVM via Online Gradient Descent
* No instante t
    * Selecionamos um vetor pt
    * Recebemos uma nova instância do dataset (yt, zt), onde yt são os atributos e zt é a classe correspondente
    * Vamos tomar uma Hinge Loss definida como max(0, 1 - zt * <pt, yt> )
    * Utilizaremos o gradiente da função de perda do instante t para calcular o próximo p(t+1)
    
Vamos também utilizar sempre pt pertencente ao espaço euclideano de norma <= 1.

A princípio o p0 pode ser definido arbitrariamente, mas como vimos descrito em alguns lugares da literatura a inicialização com o vetor nulo vamos adotar essa estratégia.
   

In [5]:
# Aqui algumas funções auxiliares úteis
def hinge_loss(p_t, z_t, y_t):
    return max(0, 1 - np.dot(p_t, y_t) * z_t )

def project_into_euclidean_norm_1( p_t ):
    norm = np.linalg.norm(p_t)
    norm_p = p_t.copy()
    if norm <= 1:
        return norm_p
    else:
        norm_p = np.true_divide( norm_p, norm )
        return norm_p

Vamos agora __encapsular__ o que foi implementado acima em um método, para poder testar diversos parâmetros de __eta__. E também vamos fazer uma função que mede a acurácia de um determinado vetor __p__ para classificar os dados.



In [6]:
def online_svm(data, eta_fnc):
    current_p = np.zeros(57)
    all_p = [ current_p ]
    cumulative_loss = 0
    # Agora vamos processar o dataset uma instancia de cada vez, num setup online
    t = 1

    for instance in data:
        instance_features = instance[0:57] # slices are semi-open intervals (:, [0, 57)
        target_class = instance[57]
        # vamos tomar a perda hinge relativo a current_p
        current_loss = hinge_loss( current_p, target_class, instance_features )
        cumulative_loss += current_loss
        # Agora vamos usar o gradiente / subgradiente para fazer a atualizacao de p_(t + 1)
        # Se hinge_loss = 0, posso usar o subgradiente definido pela reta y = 0, e não mexer em p_t

        # O caso de fato interessante é quando a perda é positiva e queremos seguir no sentido contrário do subgradiente
        if current_loss > 0:
            gradient = instance_features * (-target_class) # derivada
            current_eta = eta_fnc( t ) # Eta calculado em funcao de t, de acordo com a funcao recebida como parametro
            
            # p_t = p_(t - 1) - eta_t * ( gradiente da função de perda )
            current_p -= current_eta * gradient
            current_p = project_into_euclidean_norm_1( current_p )
            
        tmp = np.append(all_p, [current_p], axis = 0 ) # Adding current support vector to the list
        all_p = tmp.copy()
        
        t += 1 # increment in one the number of processed instances
    return all_p

Agora que encapsulamos o método em uma função que recebe como parâmetro uma função para o cálculo de eta, podemos começar a realizar alguns experimentos.

Antes disso, vou criar uma função que recebe um determinado vetor p, usa ele para __classificar__ todo o dataset e retorna a acurácia desse vetor.

In [7]:
def check_accuracy(data, classifier_vector):
    correct = 0
    total = 0
    for instance in data:
        instance_features = instance[0:57].copy()
        target_class = instance[57].copy()
        dot_prod = np.dot( instance_features, classifier_vector )
        predicted_class = -1
        if dot_prod >= 0:
            predicted_class = 1
        if target_class == predicted_class:
            correct += 1
        total += 1
    
    return ( correct / total )

def eta( t ):
    return 1 / sqrt(t)

def eta_constante( t ):
    return 0.05

In [12]:
test_data = x_scaled.copy()

results = online_svm( test_data, eta )
start_acc = check_accuracy( test_data, results[0])
end_acc = check_accuracy( test_data, results[-1])
print( start_acc, end_acc )

np.random.seed(13)
np.random.shuffle( test_data )

results2 = online_svm( test_data, eta )
start_acc2 = check_accuracy( test_data, results2[0])
end_acc2 = check_accuracy( test_data, results2[-1])
print( start_acc2, end_acc2 )

0.39404477287546186 0.6053031949576179
acuracia na iteracao 3000 = 0.606825
acuracia na iteracao 3001 = 0.606825
acuracia na iteracao 3002 = 0.606825
acuracia na iteracao 3003 = 0.606825
acuracia na iteracao 3004 = 0.606825
acuracia na iteracao 3005 = 0.606607
acuracia na iteracao 3006 = 0.606607
acuracia na iteracao 3007 = 0.606607
acuracia na iteracao 3008 = 0.606607
acuracia na iteracao 3009 = 0.606607
acuracia na iteracao 3010 = 0.606607
acuracia na iteracao 3011 = 0.606607
acuracia na iteracao 3012 = 0.606607
acuracia na iteracao 3013 = 0.606607
acuracia na iteracao 3014 = 0.606607
acuracia na iteracao 3015 = 0.606607
acuracia na iteracao 3016 = 0.606607
acuracia na iteracao 3017 = 0.606607
acuracia na iteracao 3018 = 0.606607
acuracia na iteracao 3019 = 0.606607
acuracia na iteracao 3020 = 0.606607
acuracia na iteracao 3021 = 0.606607
acuracia na iteracao 3022 = 0.606607
acuracia na iteracao 3023 = 0.606607
acuracia na iteracao 3024 = 0.606607
acuracia na iteracao 3025 = 0.606607

acuracia na iteracao 3233 = 0.605303
acuracia na iteracao 3234 = 0.605303
acuracia na iteracao 3235 = 0.605303
acuracia na iteracao 3236 = 0.605303
acuracia na iteracao 3237 = 0.605303
acuracia na iteracao 3238 = 0.605303
acuracia na iteracao 3239 = 0.605303
acuracia na iteracao 3240 = 0.605303
acuracia na iteracao 3241 = 0.605303
acuracia na iteracao 3242 = 0.605303
acuracia na iteracao 3243 = 0.605303
acuracia na iteracao 3244 = 0.605303
acuracia na iteracao 3245 = 0.605303
acuracia na iteracao 3246 = 0.605303
acuracia na iteracao 3247 = 0.605303
acuracia na iteracao 3248 = 0.605303
acuracia na iteracao 3249 = 0.605303
acuracia na iteracao 3250 = 0.605303
acuracia na iteracao 3251 = 0.605303
acuracia na iteracao 3252 = 0.605303
acuracia na iteracao 3253 = 0.605303
acuracia na iteracao 3254 = 0.605303
acuracia na iteracao 3255 = 0.605303
acuracia na iteracao 3256 = 0.605303
acuracia na iteracao 3257 = 0.605303
acuracia na iteracao 3258 = 0.605303
acuracia na iteracao 3259 = 0.605303
a

acuracia na iteracao 3462 = 0.605303
acuracia na iteracao 3463 = 0.605303
acuracia na iteracao 3464 = 0.605303
acuracia na iteracao 3465 = 0.605303
acuracia na iteracao 3466 = 0.605303
acuracia na iteracao 3467 = 0.605303
acuracia na iteracao 3468 = 0.605303
acuracia na iteracao 3469 = 0.605303
acuracia na iteracao 3470 = 0.605303
acuracia na iteracao 3471 = 0.605303
acuracia na iteracao 3472 = 0.605303
acuracia na iteracao 3473 = 0.605303
acuracia na iteracao 3474 = 0.605303
acuracia na iteracao 3475 = 0.605303
acuracia na iteracao 3476 = 0.605303
acuracia na iteracao 3477 = 0.605303
acuracia na iteracao 3478 = 0.605303
acuracia na iteracao 3479 = 0.605303
acuracia na iteracao 3480 = 0.605303
acuracia na iteracao 3481 = 0.605303
acuracia na iteracao 3482 = 0.605303
acuracia na iteracao 3483 = 0.605303
acuracia na iteracao 3484 = 0.605303
acuracia na iteracao 3485 = 0.605303
acuracia na iteracao 3486 = 0.605303
acuracia na iteracao 3487 = 0.605303
acuracia na iteracao 3488 = 0.605303
a

acuracia na iteracao 3686 = 0.605303
acuracia na iteracao 3687 = 0.605303
acuracia na iteracao 3688 = 0.605303
acuracia na iteracao 3689 = 0.605303
acuracia na iteracao 3690 = 0.605303
acuracia na iteracao 3691 = 0.605303
acuracia na iteracao 3692 = 0.605303
acuracia na iteracao 3693 = 0.605303
acuracia na iteracao 3694 = 0.605303
acuracia na iteracao 3695 = 0.605303
acuracia na iteracao 3696 = 0.605303
acuracia na iteracao 3697 = 0.605303
acuracia na iteracao 3698 = 0.605303
acuracia na iteracao 3699 = 0.605303
acuracia na iteracao 3700 = 0.605303
acuracia na iteracao 3701 = 0.605303
acuracia na iteracao 3702 = 0.605303
acuracia na iteracao 3703 = 0.605303
acuracia na iteracao 3704 = 0.605303
acuracia na iteracao 3705 = 0.605303
acuracia na iteracao 3706 = 0.605303
acuracia na iteracao 3707 = 0.605303
acuracia na iteracao 3708 = 0.605303
acuracia na iteracao 3709 = 0.605303
acuracia na iteracao 3710 = 0.605303
acuracia na iteracao 3711 = 0.605303
acuracia na iteracao 3712 = 0.605303
a

acuracia na iteracao 3908 = 0.605303
acuracia na iteracao 3909 = 0.605303
acuracia na iteracao 3910 = 0.605303
acuracia na iteracao 3911 = 0.605303
acuracia na iteracao 3912 = 0.605303
acuracia na iteracao 3913 = 0.605303
acuracia na iteracao 3914 = 0.605303
acuracia na iteracao 3915 = 0.605303
acuracia na iteracao 3916 = 0.605303
acuracia na iteracao 3917 = 0.605303
acuracia na iteracao 3918 = 0.605303
acuracia na iteracao 3919 = 0.605303
acuracia na iteracao 3920 = 0.605303
acuracia na iteracao 3921 = 0.605303
acuracia na iteracao 3922 = 0.605303
acuracia na iteracao 3923 = 0.605303
acuracia na iteracao 3924 = 0.605303
acuracia na iteracao 3925 = 0.605303
acuracia na iteracao 3926 = 0.605303
acuracia na iteracao 3927 = 0.605303
acuracia na iteracao 3928 = 0.605303
acuracia na iteracao 3929 = 0.605303
acuracia na iteracao 3930 = 0.605303
acuracia na iteracao 3931 = 0.605303
acuracia na iteracao 3932 = 0.605303
acuracia na iteracao 3933 = 0.605303
acuracia na iteracao 3934 = 0.605303
a

Qual será o __p_t__ produzido pelo Online SVM que possui maior acurácia?


In [14]:
largest_acc = [0, 0]
results_acc = []
results2_acc = []
for x in range( len(results) ):
    results_acc.append( (x, check_accuracy( test_data, results[x] ) ) )
    largest_acc[0] = max( largest_acc[0], results_acc[-1][1] )

for x in range( len( results2) ):
    results2_acc.append( (x, check_accuracy( test_data, results2[x] ) ) )
    largest_acc[1] = max( largest_acc[1], results2_acc[-1][1])
    
largest_acc

[0.8226472505976962, 0.8291675722668985]